[![AnalyticsDojo](../fig/final-logo.png)](http://rpi.analyticsdojo.com)
<center><h1>Pytorch - Revisiting Titanic</h1></center>
<center><h3><a href = 'http://rpi.analyticsdojo.com'>rpi.analyticsdojo.com</a></h3></center>


In [1]:
!pip install torch torchvision

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
import numpy as np
import pandas as pd
import pandas as pd

train= pd.read_csv('https://raw.githubusercontent.com/rpi-techfundamentals/fall2018-materials/master/input/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/rpi-techfundamentals/fall2018-materials/master/input/test.csv')



In [21]:
#Print to standard output, and see the results in the "log" section below after running your script
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#Let's look at the age field.  We can see "NaN" (which indicates missing values).s
train["Age"]

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
5       NaN
6      54.0
7       2.0
8      27.0
9      14.0
10      4.0
11     58.0
12     20.0
13     39.0
14     14.0
15     55.0
16      2.0
17      NaN
18     31.0
19      NaN
20     35.0
21     34.0
22     15.0
23     28.0
24      8.0
25     38.0
26      NaN
27     19.0
28      NaN
29      NaN
       ... 
861    21.0
862    48.0
863     NaN
864    24.0
865    42.0
866    27.0
867    31.0
868     NaN
869     4.0
870    26.0
871    47.0
872    33.0
873    47.0
874    28.0
875    15.0
876    20.0
877    19.0
878     NaN
879    56.0
880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [5]:
#Now let's recode. 
medianAge=train["Age"].median()
print ("The Median age is:", medianAge, " years old.")
train["Age"] = train["Age"].fillna(medianAge)

#Option 2 all in one shot! 
train["Age"] = train["Age"].fillna(train["Age"].median())
train["Age"] 

The Median age is: 28.0  years old.


0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
5      28.0
6      54.0
7       2.0
8      27.0
9      14.0
10      4.0
11     58.0
12     20.0
13     39.0
14     14.0
15     55.0
16      2.0
17     28.0
18     31.0
19     28.0
20     35.0
21     34.0
22     15.0
23     28.0
24      8.0
25     38.0
26     28.0
27     19.0
28     28.0
29     28.0
       ... 
861    21.0
862    48.0
863    28.0
864    24.0
865    42.0
866    27.0
867    31.0
868    28.0
869     4.0
870    26.0
871    47.0
872    33.0
873    47.0
874    28.0
875    15.0
876    20.0
877    19.0
878    28.0
879    56.0
880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
888    28.0
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [6]:
#For Recoding Data, we can use what we know of selecting rows and columns
train["Embarked"] = train["Embarked"].fillna("S")


In [7]:
# We can also use something called a lambda function 
# You can read more about the lambda function here.
#http://www.python-course.eu/lambda.php 
gender_fn = lambda x: 0 if x == 'male' else 1
train['Gender'] = train['Sex'].map(gender_fn)

In [8]:
#or we can do in one shot
train['NameLength'] = train['Name'].map(lambda x: len(x))


In [9]:

#We can start to create little small functions that will find a string.
def has_title(name):
    for s in ['Mr.', 'Mrs.', 'Miss.', 'Dr.', 'Sir.']:
        if name.find(s) >= 0:
            return True
    return False

#Now we are using that separate function in another function.  
title_fn = lambda x: 1 if has_title(x) else 0
#Finally, we call the function for name
train['Title'] = train['Name'].map(title_fn)
test['Title']= train['Name'].map(title_fn)


##Update

In [10]:

train['FamilyS']=train['SibSp'] + train['Parch'] + 1
test['FamilyS']=test['SibSp'] + test['Parch'] + 1

def family(x):
    if x < 2:
        return 'Single'
    elif x == 2:
        return 'Couple'
    elif x <= 4:
        return 'InterM'
    else:
        return 'Large'
    
train['FamilyS'] = train['FamilyS'].apply(family)
test['FamilyS'] = test['FamilyS'].apply(family)

In [11]:
#Select out only the relevant items
train_min=train.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)
train_min
test_min=test.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)
train_min


,Survived,Pclass,Sex,Age,Fare,Embarked,Gender,NameLength,Title,FamilyS
0,0,3,male,22.0,7.2500,S,0,23,1,Couple
1,1,1,female,38.0,71.2833,C,1,51,1,Couple
2,1,3,female,26.0,7.9250,S,1,22,1,Single
3,1,1,female,35.0,53.1000,S,1,44,1,Couple
4,0,3,male,35.0,8.0500,S,0,24,1,Single
5,0,3,male,28.0,8.4583,Q,0,16,1,Single
6,0,1,male,54.0,51.8625,S,0,23,1,Single
7,0,3,male,2.0,21.0750,S,0,30,0,Large
8,1,3,female,27.0,11.1333,S,1,49,1,InterM
9,1,2,female,14.0,30.0708,C,1,35,1,Couple


In [12]:
#Adding .values will make whatever comes out into a numpy array. 
X_train = train_min.iloc[:, 1:10].values
Y_train = train_min.iloc[:, 0].values
X_test = train_min.iloc[:, 1:10].values
X_train[1]

array([1, 'female', 38.0, 71.2833, 'C', 1, 51, 1, 'Couple'], dtype=object)

In [13]:
# Previously we used get dummies (part of pandas)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X_train[:, 1] = labelencoder_X_1.fit_transform(X_train[:, 1]) #Female/male
X_train[:, 4] = labelencoder_X_1.fit_transform(X_train[:, 4]) #Embarked
X_train[:, 8] = labelencoder_X_1.fit_transform(X_train[:, 8]) #Couple


labelencoder_X_2 = LabelEncoder()
X_test[:, 1] = labelencoder_X_2.fit_transform(X_test[:, 1])
X_test[:, 4] = labelencoder_X_2.fit_transform(X_test[:, 4])
X_test[:, 8] = labelencoder_X_2.fit_transform(X_test[:, 8])
X_train

array([[3, 1, 22.0, ..., 23, 1, 0],
       [1, 0, 38.0, ..., 51, 1, 0],
       [3, 0, 26.0, ..., 22, 1, 3],
       ..., 
       [3, 0, 28.0, ..., 40, 1, 1],
       [1, 1, 26.0, ..., 21, 1, 3],
       [3, 1, 32.0, ..., 19, 1, 3]], dtype=object)

In [14]:
# Converting categorical values to one-hot representation
one_hot_encoder = OneHotEncoder(categorical_features = [0, 1, 4, 7, 8])
X_train = one_hot_encoder.fit_transform(X_train).toarray()
X_test = one_hot_encoder.fit_transform(X_test).toarray()

In [15]:
#Double check no missing values
print(np.isnan(X_train).sum(),np.isnan(X_test).sum())
print(X_train[0])

0 0
[  0.     0.     1.     0.     1.     0.     0.     1.     0.     1.     1.
   0.     0.     0.    22.     7.25   0.    23.  ]


In [16]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.2)

In [19]:
#import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(19, 270)
        self.fc2 = nn.Linear(270, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        
        return x
    
net = Net()

ModuleNotFoundError: No module named 'torch'

In [18]:
#Writing to File
submission=pd.DataFrame(test.loc[:,['PassengerId','Survived']])

#Any files you save will be available in the output tab below
submission.to_csv('submission.csv', index=False)